# Trabajo Práctico N° 2
## Objetivo:

Para cada dispositivo presentado por Jampp, determinar el tiempo que transcurrirá hasta que el mismo aparezca nuevamente en una subasta, y el tiempo hasta que el usuario del mismo decida instalar una nueva aplicación.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import auc, accuracy_score
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold, RandomizedSearchCV, train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
import xgboost as xgb
import datetime as dt

## Función para leer "n" lineas del archivo "auctions"

In [3]:
import gzip
def ggzip(source_filepath, dest_filepath, block_size=65536):
    with gzip.open(dest_filepath, 'wb') as d_file, \
            open(source_filepath, 'rb') as s_file:
        for i in range(block_size ):
            line = s_file.readline()
            if not line:
                break
            else:
                d_file.write(line)
        #d_file.write(block)
        
ggzip("data/auctions.csv.gzip", "data/auctions_reducido.csv.gzip", 1000000)

In [4]:
auct = pd.read_csv("data/auctions_reducido.csv.gzip", compression = 'gzip' )

In [10]:
auct.count()

date           999999
device_id      999999
ref_type_id    999999
source_id      999999
dtype: int64

In [12]:
auct.head()

,date,device_id,ref_type_id,source_id
0,2019-04-23 18:58:00.842116,2564673204772915246,1,0
1,2019-04-23 18:58:01.530771,4441121667607578179,7,0
2,2019-04-23 18:58:01.767562,7721769811471055264,1,0
3,2019-04-23 18:58:02.363468,6416039086842158968,1,0
4,2019-04-23 18:58:02.397559,1258642015983312729,1,0


## Leo todos los datos para tomar ventanas de 3 días

In [ ]:
auct = pd.read_csv("data/auctions.csv.gzip",dtype = {"ref_type_id": np.int8, "source_id": np.int8})
#evts = pd.read_csv("data/events.csv.gzip", dtype = {"event_id": np.int16, "application_id": np.int16})
#clks = pd.read_csv("data/clicks.csv.gzip", dtype = {"advertiser_id": np.int8, "action_id": np.float16, "source_id": np.int8, "carrier_id": np.float16, "brand": np.float16, "touchX": np.float16, "touchY": np.float16})

In [9]:
auct.head()

,date,device_id,ref_type_id,source_id
0,2019-04-23 18:58:00.842116,2564673204772915246,1,0
1,2019-04-23 18:58:01.530771,4441121667607578179,7,0
2,2019-04-23 18:58:01.767562,7721769811471055264,1,0
3,2019-04-23 18:58:02.363468,6416039086842158968,1,0
4,2019-04-23 18:58:02.397559,1258642015983312729,1,0


In [2]:
evts = pd.read_csv("data/events.csv.gzip", dtype = {"index": np.int32, "event_id": np.int16, "application_id": np.int16})

/home/sol/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3044: DtypeWarning: Columns (13,20) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
evts.head(1)

,index,date,event_id,ref_type,ref_hash,application_id,attributed,device_countrycode,device_os_version,device_brand,...,trans_id,user_agent,event_uuid,carrier,kind,device_os,wifi,connection_type,ip_address,device_language
0,2130678,2019-04-20 01:42:49.120,0,1891515180541284343,5857744372586891366,210,False,6287817205707153877,NaN,NaN,...,NaN,5.046185e+18,5b506964-5f47-4b28-a8c2-8a92d6c23379,NaN,5.882882e+18,NaN,False,NaN,7544543351571901618,3.301378e+18


In [5]:
evts.dtypes

index                   int64
date                   object
event_id                int16
ref_type                int64
ref_hash                int64
application_id          int16
attributed               bool
device_countrycode      int64
device_os_version     float64
device_brand          float64
device_model          float64
device_city           float64
session_user_agent    float64
trans_id               object
user_agent            float64
event_uuid             object
carrier               float64
kind                  float64
device_os             float64
wifi                     bool
connection_type        object
ip_address              int64
device_language       float64
dtype: object

# Armo los sets de datos por ventanas de 3 días

## Ventana N° 1: 18 al 20 de abril

In [13]:
ventana1 = auct.loc[auct["date"].str.contains('2019-04-18') | auct["date"].str.contains('2019-04-19') | auct["date"].str.contains('2019-04-20'), :]

In [15]:
ventana1.to_csv(path_or_buf = 'data/auctions_ventana1.csv', index = False)

In [8]:
evts_vent1 = evts.loc[evts["date"].str.contains('2019-04-18') | evts["date"].str.contains('2019-04-19') | evts["date"].str.contains('2019-04-20'), :]

In [9]:
evts_vent1.to_csv(path_or_buf = 'data/events_ventana1.csv', index = False)

## Ventana N° 1: 19 al 21 de abril

In [16]:
ventana2 = auct.loc[auct["date"].str.contains('2019-04-19') | auct["date"].str.contains('2019-04-20') | auct["date"].str.contains('2019-04-21'), :]

In [17]:
ventana2.to_csv(path_or_buf = 'data/auctions_ventana2.csv', index = False)

In [11]:
evts_vent2 = evts.loc[evts["date"].str.contains('2019-04-19') | evts["date"].str.contains('2019-04-20') | evts["date"].str.contains('2019-04-21'), :]

In [12]:
evts_vent2.to_csv(path_or_buf = 'data/events_ventana2.csv', index = False)

## Ventana N° 1: 20 al 22 de abril

In [18]:
ventana3 = auct.loc[auct["date"].str.contains('2019-04-20') | auct["date"].str.contains('2019-04-21') | auct["date"].str.contains('2019-04-22'), :]

In [19]:
ventana3.to_csv(path_or_buf = 'data/auctions_ventana3.csv', index = False)

In [13]:
evts_vent3 = evts.loc[evts["date"].str.contains('2019-04-20') | evts["date"].str.contains('2019-04-21') | evts["date"].str.contains('2019-04-22'), :]

In [14]:
evts_vent3.to_csv(path_or_buf = 'data/events_ventana3.csv', index = False)

## Ventana N° 1: 21 al 23 de abril

In [20]:
ventana4 = auct.loc[auct["date"].str.contains('2019-04-21') | auct["date"].str.contains('2019-04-22') | auct["date"].str.contains('2019-04-23'), :]

In [21]:
ventana4.to_csv(path_or_buf = 'data/auctions_ventana4.csv', index = False)

In [15]:
evts_vent4 = evts.loc[evts["date"].str.contains('2019-04-21') | evts["date"].str.contains('2019-04-22') | evts["date"].str.contains('2019-04-23'), :]

In [16]:
evts_vent4.to_csv(path_or_buf = 'data/events_ventana4.csv', index = False)

## Ventana N° 1: 22 al 24 de abril

In [22]:
ventana5 = auct.loc[auct["date"].str.contains('2019-04-22') | auct["date"].str.contains('2019-04-23') | auct["date"].str.contains('2019-04-24'), :]

In [23]:
ventana5.to_csv(path_or_buf = 'data/auctions_ventana5.csv', index = False)

In [17]:
evts_vent5 = evts.loc[evts["date"].str.contains('2019-04-22') | evts["date"].str.contains('2019-04-23') | evts["date"].str.contains('2019-04-24'), :]

In [18]:
evts_vent5.to_csv(path_or_buf = 'data/events_ventana5.csv', index = False)